# Rfam investigation to convert alignment to structures

At a high-level, secondary structure of an aligned sequence can be obtained by projecting the consensus structure to the aligned sequence. However, details on exactly how this is done is a bit of a question.

Here, structures obtained from a custom script is compared against ones from R2DT

In [32]:
import re
regex = re.compile(r'[a-zA-Z_\,-\.]')


def get_structure(seq_align: str, consensus_st: str):
    assert len(seq_align) == len(consensus_st)

    seq, structure = [], []
    for base, st_char in zip(seq_align, consensus_st):
        if base == '-':
            pass
        elif base in ['A', 'C', 'G', 'U']:
            seq.append(base)
            structure.append(st_char)
        else:
            raise ValueError(f"Invalid base {base}")

    seq = ''.join(seq)
    structure = ''.join(structure)

    return seq, structure

def wuss_to_db(structure: str):
    db_st = []
    for i in structure:
        if i in ['(', '<', '[']:
            db_st.append('(')
        elif i in [')', '>', ']']:
            db_st.append(')')
        elif re.match(regex, i):
            db_st.append('.')

    db_st = ''.join(db_st)

    assert len(db_st) == len(structure)

    return db_st

In [36]:
# Test cases from RF03160
# Stockholm file from https://rfam.org/family/RF03160/alignment?acc=RF03160&format=stockholm&download=0
cons_struct = '((((.---.((((BB<<<<<<__.................._____........................................................_>>>>>><<<__AAA__bb._>>>................,,,,,........................)))).--aaa....))))'

print('URS0000D6C384_12908/1-78')
seq_align = 'UUUU-UAA-CCCAGCCACUAGCAUUGACA------------CUU---------------------------------------------------UGUC-GUGUUCGUGCCGGUCCCAAGC-CCGG----------------AGAAAA-----------------------UGGG-GAGGU----UUUU'
seq, structure = get_structure(seq_align, cons_struct)
assert seq == 'UUUUUAACCCAGCCACUAGCAUUGACACUUUGUCGUGUUCGUGCCGGUCCCAAGCCCGGAGAAAAUGGGGAGGUUUUU'
target_st = '((((...((((..((((((..................))))))(((..........)))......)))).....))))'
if wuss_to_db(structure) != target_st:
    print('Inconsistent structure')
    print(wuss_to_db(structure))
    print(target_st)

print('URS0000D6AF4C_6183/1-72')
seq_align = 'CUCU-CAA-CUCCGCCUGUAGCUCC----------------UCCG------------------------------------------------------GGGGUUACUGCCGGUCCCAAGC-CCGG----------------GUAAA------------------------GGAG-GAGGGU---CGGG'
seq, structure = get_structure(seq_align, cons_struct)
assert seq == 'CUCUCAACUCCGCCUGUAGCUCCUCCGGGGGUUACUGCCGGUCCCAAGCCCGGGUAAAGGAGGAGGGUCGGG'
target_st = '.(((...((((..((((((............))))))(((..........))).....)))).......)))'
if wuss_to_db(structure) != target_st:
    print('Inconsistent structure')
    print(wuss_to_db(structure))
    print(target_st)

# URS0000D6C49D_12908/29-85
# Alignment does not exist in RNAcentral
# seq_align = 'GCAC-UAA-UGUAGC--------------------------UCAGA---------------------------------------------------------------CCUGUGACAAGC-CAAG----------------GCUAGAAAAA-------------------UACA-GAGUC----GUGC'
# seq, structure = get_structure(seq_align, cons_struct)
# assert seq == 'CUCUCAACUCCGCCUGUAGCUCCUCCGGGGGUUACUGCCGGUCCCAAGCCCGGGUAAAGGAGGAGGGUCGGG'
# target_st = '.(((...((((..((((((............))))))(((..........))).....)))).......)))'
# if wuss_to_db(structure) != target_st:
#     print('Inconsistent structure')
#     print(wuss_to_db(structure))
#     print(target_st)

print('URS0000D669BF_12908/1-56')
seq_align = 'GGCC-UAA-UGCAGC--------------------------AUAGU---------------------------------------------------------------CCUGUCACAAGC-CAGG----------------CUGAAAAA---------------------UGCA-GAGUGA---GGCA'
seq, structure = get_structure(seq_align, cons_struct)
assert seq == 'GGCCUAAUGCAGCAUAGUCCUGUCACAAGCCAGGCUGAAAAAUGCAGAGUGAGGCA'
target_st = '((((...((((.......(((..........)))........))))......))))'
if wuss_to_db(structure) != target_st:
    print('Inconsistent structure')
    print(wuss_to_db(structure))
    print(target_st)

print('URS0000D6C67E_12908')
seq_align = 'CUCC-UAA-UGCAGC--------------------------CGAAG---------------------------------------------------------------GCGGUCACAAGC-CCGA----------------UUGAGAGA---------------------UGCA-GAGUG----GGAA'
seq, structure = get_structure(seq_align, cons_struct)
assert seq == 'CUCCUAAUGCAGCCGAAGGCGGUCACAAGCCCGAUUGAGAGAUGCAGAGUGGGAA'
target_st = '((((...((((.......(((..........)))........)))).....))))'
if wuss_to_db(structure) != target_st:
    print('Inconsistent structure')
    print(wuss_to_db(structure))
    print(target_st)

URS0000D6C384_12908/1-78
URS0000D6AF4C_6183/1-72
Inconsistent structure
((((...((((..((((((............))))))(((..........))).....))))......))))
.(((...((((..((((((............))))))(((..........))).....)))).......)))
URS0000D669BF_12908/1-56
URS0000D6C67E_12908


Many of the sequences in the alignment file actually does not have pre-computed structures in RNAcentral. There are also few cases where the aligned sequences do not exist in RNAcentral